<a href="https://colab.research.google.com/github/olivia-maras/olivia-maras/blob/main/Copy_of_4_AugmentingDataAndBasicAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Augmenting Data in Python with Pandas

One of the key features of the pandas library is that it allows us streamline in-memory manipulation of data, which can be especially useful when we want to augment our data set with information from another source.

For example, while our current source data on vaccinations contains many useful elements, we might want to add information about employment, or income. Pandas lets us join data sets based on one or more columns, using different styles of joins. It also adds flag columns each time (if we choose), so that we can keep track of what has and has not been matched.

In [ ]:
# first, import the pandas library, giving it a nickname of "pd" for short
import pandas as pd
import numpy as np

In [ ]:
# THIS CODE REQUIRED FOR GOOGLE COLAB
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
# Documentation found here: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# THIS CODE REQUIRED FOR GOOGLE COLAB
# Link to data file stored in Drive: https://drive.google.com/file/d/10P11wZOuwlCN9krxokxmeryOK3mnMZby/view?usp=sharing
file_id = '10P11wZOuwlCN9krxokxmeryOK3mnMZby' # notice where this string comes from in link above

imported_file = drive.CreateFile({'id': file_id}) # creating an accessible copy of the shared data file
print(imported_file['title'])  # it should print the title of desired file
imported_file.GetContentFile(imported_file['title']) # refer to it in this notebook by the same name as it has in Drive

owid-covid-data.csv


In [ ]:
# our data is stored as a csv, so we'll use the `read_csv()` method.
# similar methods exist for other data formats, e.g. `read_excel()` or `read_stata()`
# for a complete list of these methods, see https://pandas.pydata.org/docs/reference/io.html

# we're going to take care to name our dataframe and other variables descriptively
# this will make reading our code later much more intuitive
vaccine_data = pd.read_csv('owid-covid-data.csv')

In [ ]:
# Importing some additional data to join with our current data set
# Link to data file stored in Drive: https://drive.google.com/file/d/1JqF_AqulFQCeVn1eLCh_mHmBAI7Bw6WL/view?usp=sharing
file_id = '1JqF_AqulFQCeVn1eLCh_mHmBAI7Bw6WL' # notice where this string comes from in link above

imported_file = drive.CreateFile({'id': file_id}) # creating an accessible copy of the shared data file
print(imported_file['title'])  # it should print the title of desired file
imported_file.GetContentFile(imported_file['title']) # refer to it in this notebook by the same name as it has in Drive

wb_employment_data.csv


In [ ]:
# now, let's bring in some employment data and see what it contains

unemp_data = pd.read_csv('wb_employment_data.csv')
print(unemp_data.head())

  Country Name Country Code  \
0        Aruba          ABW   
1  Afghanistan          AFG   
2       Angola          AGO   
3      Albania          ALB   
4      Andorra          AND   

                                      Indicator Name  Indicator Code  1960  \
0  Unemployment, total (% of total labor force) (...  SL.UEM.TOTL.ZS   NaN   
1  Unemployment, total (% of total labor force) (...  SL.UEM.TOTL.ZS   NaN   
2  Unemployment, total (% of total labor force) (...  SL.UEM.TOTL.ZS   NaN   
3  Unemployment, total (% of total labor force) (...  SL.UEM.TOTL.ZS   NaN   
4  Unemployment, total (% of total labor force) (...  SL.UEM.TOTL.ZS   NaN   

   1961  1962  1963  1964  1965  ...   2011   2012   2013       2014  \
0   NaN   NaN   NaN   NaN   NaN  ...    NaN    NaN    NaN        NaN   
1   NaN   NaN   NaN   NaN   NaN  ...  11.51  11.52  11.54  11.450000   
2   NaN   NaN   NaN   NaN   NaN  ...   7.36   7.37   7.38   7.310000   
3   NaN   NaN   NaN   NaN   NaN  ...  13.48  13.38  15.8

In [ ]:
# and let's remind ourselves of the columns in our vaccine_data
print(vaccine_data.head())

  iso_code continent     location        date  total_cases  new_cases  \
0      AFG      Asia  Afghanistan  2020-02-24          5.0        5.0   
1      AFG      Asia  Afghanistan  2020-02-25          5.0        0.0   
2      AFG      Asia  Afghanistan  2020-02-26          5.0        0.0   
3      AFG      Asia  Afghanistan  2020-02-27          5.0        0.0   
4      AFG      Asia  Afghanistan  2020-02-28          5.0        0.0   

   new_cases_smoothed  total_deaths  new_deaths  new_deaths_smoothed  ...  \
0                 NaN           NaN         NaN                  NaN  ...   
1                 NaN           NaN         NaN                  NaN  ...   
2                 NaN           NaN         NaN                  NaN  ...   
3                 NaN           NaN         NaN                  NaN  ...   
4                 NaN           NaN         NaN                  NaN  ...   

   female_smokers  male_smokers  handwashing_facilities  \
0             NaN           NaN        

In [ ]:
# since unemployment is a "lagging" indicator, we're probably not as interested in the unemployment rate in 
# 2020 as the unemployment rate in 2019: if unemployment was high then, it may be a better predictor of a 
# country's economic state in 2020

# first, let's get *just* the unemployment rates for 2019

unemp_2019 = unemp_data.loc[:,['Country Code','2019']]
print(unemp_2019)

    Country Code       2019
0            ABW        NaN
1            AFG  10.980000
2            AGO   6.930000
3            ALB  11.470000
4            AND        NaN
..           ...        ...
259          XKX        NaN
260          YEM  12.900000
261          ZAF  28.469999
262          ZMB  11.910000
263          ZWE   5.020000

[264 rows x 2 columns]


In [ ]:
# now, let's merge the unemployment data with our vaccine data

new_data = vaccine_data.merge(unemp_2019, how='left', left_on='iso_code', right_on='Country Code')
print(new_data.head().transpose())

                                                   0            1  \
iso_code                                         AFG          AFG   
continent                                       Asia         Asia   
location                                 Afghanistan  Afghanistan   
date                                      2020-02-24   2020-02-25   
total_cases                                      5.0          5.0   
...                                              ...          ...   
excess_mortality_cumulative                      NaN          NaN   
excess_mortality                                 NaN          NaN   
excess_mortality_cumulative_per_million          NaN          NaN   
Country Code                                     AFG          AFG   
2019                                           10.98        10.98   

                                                   2            3            4  
iso_code                                         AFG          AFG          AFG  
continent

In [ ]:
# of course, now we have essentially a duplicate column, so let's get rid of that
# note that in pandas, if there is an "axis" option, 0=rows and 1=columns; 0 is usually the default
# there is also the more intuitive option: new_data.drop(columns=['Country Code'])

trimmed_data = new_data.drop(['Country Code'], axis=1)
print(trimmed_data.head().transpose())

                                                   0            1  \
iso_code                                         AFG          AFG   
continent                                       Asia         Asia   
location                                 Afghanistan  Afghanistan   
date                                      2020-02-24   2020-02-25   
total_cases                                      5.0          5.0   
...                                              ...          ...   
excess_mortality_cumulative_absolute             NaN          NaN   
excess_mortality_cumulative                      NaN          NaN   
excess_mortality                                 NaN          NaN   
excess_mortality_cumulative_per_million          NaN          NaN   
2019                                           10.98        10.98   

                                                   2            3            4  
iso_code                                         AFG          AFG          AFG  
continent

In [ ]:
# to make our grabbing the data for a specific country easier, let's write a function!

def get_country_data(df, country_code):
  df_copy = df.copy()
  return df_copy.loc[df_copy['iso_code']==country_code]

In [ ]:
# and now let's just grab the data for Australia as an example
just_AUS = get_country_data(trimmed_data, 'AUS')

## One calculation, three ways: Finding the mean of a column with pandas

While Python, pandas and numpy all contain lots of useful "helper" functions for doing basic calculations, it's often worth exploring different ways of completing the same task. Sometimes one method will execute more quickly than another (which can become very important when working with a lot of data). At other times, the default operation of a helper function might not be quite what we're looking for, and so knowing another method for calculating, say, the median or different quartiles can be valuable. 

Here, we'll look at three different ways to find the mean: first, by "manually" calculating the mean by adding up all the values and dividing by the number of values, then by using numpy's built-in mean() function, then by using the one built-in to pandas. Along the way we'll confirm whether or not we get the same values in each instance, and we'll also try timing them to see which one is the fastest for us!

For this exercise, we'll look at the new cases each month in Australia, and calculate the average.

In [ ]:
# if we're going to calculate the average, we first have to make sure we don't have
# any "NaN" values; let's drop them

# if we don't include the "inplace" parameter, our underlyiing DataFrame won't be changed
just_AUS.dropna(subset=['new_cases'], inplace=True)

In [ ]:
# uh-oh. What happened?
# well, when we wrote our `get_country_data` function, we just used square brackets, 
# rather that the `.loc` method. As a result, the `just_AUS` DataFrame is only a "view"
# of our larger DataFrame, rather than a true copy. Let's fix it.

In [ ]:
# if we want to do this "manually," we'll first need to pull out the parameter of interest as its own list
# fortunately, this is quite simple thanks to the `tolist()` method

new_cases_list = just_AUS['new_cases'].tolist()

all_cases = 0

for case_count in new_cases_list:
    all_cases += case_count

monthly_avg = all_cases/len(new_cases_list)

print(monthly_avg)

9701.090100111234


In [ ]:
# we can also calculate the mean on the list using numpy functions

monthly_avg_np = np.mean(new_cases_list)

print(monthly_avg_np)

9701.090100111234


In [ ]:
# or we can use the pandas function on the column of our new DataFrame

monthly_avg_pd = just_AUS['new_cases'].mean()

print(monthly_avg_pd)

9701.090100111234


In [ ]:
# now let's compare how long each method takes, using the `timeit` "magic" command

def manual_avg(data_list):
  all_cases = 0

  for case_count in data_list:
      all_cases += case_count

  monthly_avg = all_cases/len(data_list)

%timeit manual_avg(new_cases_list)


10000 loops, best of 5: 37.4 µs per loop


In [ ]:
%timeit np.mean(new_cases_list)

10000 loops, best of 5: 95.2 µs per loop


In [ ]:
%timeit just_AUS['new_cases'].mean()

The slowest run took 7.14 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 67.8 µs per loop


We won't dig into all the reasons for these differences right now, but just know that it's something you may want to check if you have very large calculations to perform!